In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor 

In [38]:
df=pd.read_csv(r"../data/attrition_data.csv")
df=df.replace({'STATUS': {'T': 1, 'A': 0}})
df.drop(['TERMINATION_YEAR'], axis = 1, inplace=True)

In [39]:
column_values = df['JOBCODE'].values.ravel()
unique_values = pd.unique(column_values)
len(unique_values)

1278

In [40]:
df.describe()

,EMP_ID,ANNUAL_RATE,HRLY_RATE,JOBCODE,JOB_SATISFACTION,AGE,PERFORMANCE_RATING,STATUS,PREVYR_1,PREVYR_2,PREVYR_3,PREVYR_4,PREVYR_5,A,B,C,D,E,experience
count,8.611000e+03,8.611000e+03,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000,8611.000000
mean,5.171513e+09,8.911329e+04,49.828475,51407.488793,2.762978,40.157357,3.006387,0.482406,1.440483,1.141099,0.865753,0.689351,0.520381,0.593659,0.471374,0.363140,0.290210,0.226803,2.945186
std,2.396210e+09,5.892069e+04,28.379991,22912.299967,1.412292,13.735217,1.408727,0.499719,1.298630,1.308223,1.225873,1.143279,1.012074,0.491178,0.499209,0.480933,0.453886,0.418788,1.939208
min,1.000475e+09,1.678600e+04,14.000000,10006.000000,1.000000,18.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3.105552e+09,5.051600e+04,31.000000,33520.000000,2.000000,28.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,5.227003e+09,7.362700e+04,42.000000,52630.000000,3.000000,39.000000,3.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,7.256871e+09,1.086390e+05,59.000000,69401.000000,4.000000,52.000000,4.000000,1.000000,3.000000,2.000000,2.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,5.000000
max,9.982663e+09,1.250924e+06,608.000000,99793.000000,5.000000,64.000000,5.000000,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000


In [41]:
job_group_combiner = {'Plant & Facilities Maintenance':'Manufacturing & Production', 'Physical Flows':'Manufacturing & Production', 'Production & Operations':'Manufacturing & Production', 'Industrial Quality':'Manufacturing & Production', 'Package Development':'Manufacturing & Production', 'Logistics - Manufacturing':'Manufacturing & Production', 'Plant Management':'Manufacturing & Production','Manufacturing Supply Chain':'Supply Chain','Distribution/Administration':'Supply Chain','Supply Chain Finance':'Supply Chain','Market Supply Logistics':'Supply Chain','Corporate Supply Chain':'Supply Chain','Transportation & Warehousing':'Supply Chain','Prod Planning & Inventory Ctl':'Supply Chain','Logistics - Distribution':'Supply Chain','Demand Planning':'Supply Chain','Supply Chain Administration':'Supply Chain','Finance':'Finance','Tax':'Finance','Accounting':'Finance','Flows & Sub-Contracting':'Finance','Accounts Payable':'Finance','Treasury':'Finance','Insurance & Risk Management':'Finance','Supply Chain Administration':'Business','R&I Evaluation':'Business','Public Relations':'Business','Customer Care':'Business','Multi-Channel':'Business','R&I Safety Evaluation':'Business','Customer Relationship Mgmt':'Marketing','Marketing - Direct':'Marketing','Promotional Purchasing':'Marketing','Marketing Support/Services':'Marketing','Market Research':'Marketing','Marketing - Global':'Marketing','Integrated Mktg Communications':'Marketing','Brand Operations':'Marketing','Social Media':'Marketing','Integrated Marketing Comm':'Marketing','General Administration':'General','General Management':'General','R&I General Management':'General','R&I Development/Pre-Develpmnt':'Research & Development','Creative Service/Copy':'Research & Development','Engineering':'Research & Development','Technical Packaging':'Research & Development','Quality Assurance':'Research & Development','Digital':'Research & Development','Analytical/Microbiology':'Research & Development','eCommerce':'Research & Development','Demi-Grand':'Research & Development','Applied Research':'Research & Development','Advanced Research':'Research & Development','Web':'Research & Development','IT Business Applications':'IT','IT Technologies and Infrstrctr':'IT','IT Governance and Management':'IT','IT Architecture and Integrtion':'IT','IT Digital':'IT','IT Security/Risk and Quality':'IT','Sourcing':'Human Resources','Human Resources':'Human Resources','Legal':'Legal','Intellectual Proprty & Patents':'Legal','Regulatory Affairs':'Legal','Claims Substantiation':'Legal'}

In [42]:
df['COMBINED_JOB_GROUP'] = [job_group_combiner[job_group] for job_group in df['JOB_GROUP']]

In [43]:
df['COMBINED_JOB_GROUP'].describe()

count                           8611
unique                            10
top       Manufacturing & Production
freq                            2740
Name: COMBINED_JOB_GROUP, dtype: object

In [44]:
quantiled_annual_rate = df['ANNUAL_RATE'].quantile([0.25,0.5,0.75])
discretized_annual_rate_list = []
for annual_rate in df['ANNUAL_RATE']:
    if quantiled_annual_rate[0.25] > annual_rate:
        discretized_annual_rate_list.append('LOW')
    elif quantiled_annual_rate[0.25] <= annual_rate and quantiled_annual_rate[0.50] > annual_rate:
        discretized_annual_rate_list.append('MEDIUM')
    elif quantiled_annual_rate[0.50] <= annual_rate and quantiled_annual_rate[0.75] > annual_rate:
        discretized_annual_rate_list.append('HIGH')
    elif quantiled_annual_rate[0.75] <= annual_rate:
        discretized_annual_rate_list.append('VERY HIGH')
df['DISCRETIZED_ANNUAL_RATE'] = discretized_annual_rate_list

In [45]:
df.to_csv(r'../data/created.csv', index = False)

In [46]:
x=df.drop(['EMP_ID', 'ANNUAL_RATE', 'HRLY_RATE', 'JOB_GROUP', 'STATUS','HIRE_MONTH','A', 'B', 'C', 'D', 'E'], axis=1)
y=df['STATUS']

In [47]:
x = pd.get_dummies(x, columns=['COMBINED_JOB_GROUP', 'ETHNICITY', 'SEX', 'MARITAL_STATUS', 'NUMBER_OF_TEAM_CHANGED', 'REFERRAL_SOURCE', 'IS_FIRST_JOB', 'TRAVELLED_REQUIRED', 'DISABLED_EMP', 'DISABLED_VET'])

In [48]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
columns_to_be_encoded = ['JOB_SATISFACTION','EDUCATION_LEVEL','PREVYR_1','PREVYR_2','PREVYR_3','PREVYR_4','PREVYR_5','experience','DISCRETIZED_ANNUAL_RATE']
for column in columns_to_be_encoded:
    x[column] = label_encoder.fit_transform(x[column])

In [49]:
#from sklearn.preprocessing import LabelEncoder
#x['COMBINED_JOB_GROUP'] = label_encoder.fit_transform(x['COMBINED_JOB_GROUP'])

In [50]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [51]:
rf_model = RandomForestRegressor(max_leaf_nodes = 40, random_state=0) #76% accuracy
#rf_model = RandomForestRegressor(random_state=5,max_depth=15,n_estimators=150)
rf_model.fit(x_train, y_train)
y_pred = rf_model.predict(x_test)

/home/farid/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [52]:
print (accuracy_score(y_pred.round(),y_test))

0.7430340557275542


In [53]:
results=pd.DataFrame()
results['columns']=x_train.columns
results['importances'] = rf_model.feature_importances_
results.sort_values(by='importances',ascending=False,inplace=True)

results[:20]

,columns,importances
11,experience,0.347765
0,JOBCODE,0.242305
19,COMBINED_JOB_GROUP_Manufacturing & Production,0.151411
6,PREVYR_1,0.083677
12,DISCRETIZED_ANNUAL_RATE,0.067397
2,AGE,0.015974
15,COMBINED_JOB_GROUP_General,0.015558
9,PREVYR_4,0.009877
20,COMBINED_JOB_GROUP_Marketing,0.007926
8,PREVYR_3,0.007142


In [56]:
x=df.drop(['EMP_ID', 'ANNUAL_RATE', 'HRLY_RATE', 'STATUS', 'JOB_GROUP','A', 'B', 'C', 'D', 'E'], axis=1)
y=df['STATUS']

In [57]:
x = pd.get_dummies(x, columns=['HIRE_MONTH', 'COMBINED_JOB_GROUP', 'ETHNICITY', 'SEX', 'MARITAL_STATUS', 'NUMBER_OF_TEAM_CHANGED', 'REFERRAL_SOURCE', 'IS_FIRST_JOB', 'TRAVELLED_REQUIRED', 'DISABLED_EMP', 'DISABLED_VET'])

In [58]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
columns_to_be_encoded = ['JOB_SATISFACTION','EDUCATION_LEVEL','PREVYR_1','PREVYR_2','PREVYR_3','PREVYR_4','PREVYR_5','experience','DISCRETIZED_ANNUAL_RATE']
for column in columns_to_be_encoded:
    x[column] = label_encoder.fit_transform(x[column])

In [59]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [60]:
from sklearn.tree import DecisionTreeClassifier
np.random.seed(111)
clf = DecisionTreeClassifier(random_state=0).fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_pred,y_test))

0.6749226006191951


In [61]:
results=pd.DataFrame()
results['columns']=x_train.columns
results['importances'] = clf.feature_importances_
results.sort_values(by='importances',ascending=False,inplace=True)

results[:20]

,columns,importances
0,JOBCODE,0.186389
11,experience,0.136863
2,AGE,0.066125
31,COMBINED_JOB_GROUP_Manufacturing & Production,0.053455
4,PERFORMANCE_RATING,0.032947
12,DISCRETIZED_ANNUAL_RATE,0.032904
5,EDUCATION_LEVEL,0.031511
6,PREVYR_1,0.022063
1,JOB_SATISFACTION,0.021796
7,PREVYR_2,0.015959
